CO653 - Learning Machines and Intelligent Agents

Team mates:

Muneef - 22206529

Entwan - 22135347


In [2]:
#Importing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.utils import resample
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train = pd.read_csv('/content/drive/MyDrive/Loan_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Loan_test.csv')

train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = train.isnull().mean() * 100

# Display columns with more than 0% missing values
print(missing_percentage[missing_percentage > 0])

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = test.isnull().mean() * 100

# Display columns with more than 0% missing values
print(missing_percentage[missing_percentage > 0])

In [ ]:
missing = train[train.isnull().any(axis=1)]
missing

In [ ]:
def handle_missing_values(data, threshold=0.5):
    """
    Handles missing values in a dataset:
    - Drops columns with more than a specified threshold of missing values.
    - Fills remaining missing values with median for numeric columns and mode for categorical columns.

    Parameters:
        data (pd.DataFrame): The input dataset.
        threshold (float): The proportion of missing values above which columns are dropped (default 0.5).

    Returns:
        pd.DataFrame: The cleaned dataset.
    """
    # Step 2: Fill numeric columns with median
    numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_columns:
        if data[col].isnull().sum() > 0:  # Only fill if there are missing values
            data[col] = data[col].fillna(data[col].median())

    # Step 3: Fill categorical columns with mode
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns
    for col in categorical_columns:
        if data[col].isnull().sum() > 0:  # Only fill if there are missing values
            data[col] = data[col].fillna(data[col].mode()[0])  # Use the most frequent value

    return data

# Example usage
train = handle_missing_values(train, threshold=0.5)

# Check the result
print(train.isnull().sum())

In [ ]:
# Example usage
test = handle_missing_values(test, threshold=0.5)

# Check the result
print(train.isnull().sum())

In [ ]:
train.head(10)

In [ ]:
train = train.drop("Loan_ID", axis = 1)

In [ ]:
test = test.drop("Loan_ID", axis = 1)

In [ ]:
# Get categorical columns present in the current DataFrame
categorical_columns = train.select_dtypes(include=['object', 'category']).columns

label_encoder = LabelEncoder()
# Dictionary to store mappings
label_mappings_train = {}


for column in categorical_columns:
    train[column] = label_encoder.fit_transform(train[column])
    label_mappings_train[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [ ]:
# Get categorical columns present in the current DataFrame
categorical_columns = test.select_dtypes(include=['object', 'category']).columns


label_mappings_test = {}

for column in categorical_columns:
    test[column] = label_encoder.fit_transform(test[column])
    label_mappings_test[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [ ]:
# Print the mappings
for col, mapping in label_mappings_train.items():
    print(f"Column: {col}")
    print(mapping)
    print()

In [ ]:
# Print the mappings
for col, mapping in label_mappings_test.items():
    print(f"Column: {col}")
    print(mapping)
    print()

In [ ]:
train.head(15)

In [ ]:
test.head()

In [ ]:
train["ApplicantIncome"].describe()

In [ ]:
train["LoanAmount"].describe()

In [ ]:
'ApplicantIncome	CoapplicantIncome	LoanAmount'

sns.boxplot(train["ApplicantIncome"])

In [ ]:
'ApplicantIncome	CoapplicantIncome	LoanAmount'

sns.boxplot(train["CoapplicantIncome"])

In [ ]:
'ApplicantIncome	CoapplicantIncome	LoanAmount'

sns.boxplot(train["LoanAmount"])

In [ ]:
unique = train["Loan_Amount_Term"].unique()
unique

In [ ]:
# Define the numerical columns that need scaling
high_num_cols = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term"]

# Create a copy of the DataFrame (optional, to avoid modifying the original)
train_scaled = train.copy()
test_scaled = train.copy()

# Apply StandardScaler only to the selected numerical columns
scaler = StandardScaler()
train_scaled[high_num_cols] = scaler.fit_transform(train_scaled[high_num_cols])
test_scaled[high_num_cols] = scaler.fit_transform(test_scaled[high_num_cols])

# Now df_scaled has the scaled numerical features while categorical ones remain unchanged


In [ ]:
train_scaled.head()

In [ ]:
# Plot categorical feature distributions
for col in categorical_columns:
    plt.figure(figsize=(8, 5))
    sns.countplot(y=train_scaled[col], order=train[col].value_counts().index)
    plt.title(f"Distribution of {col}")
    plt.xlabel("Count")
    plt.ylabel(col)
    plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_scaled.corr(), annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
sns.pairplot(train_scaled, hue="Loan_Status")
plt.show()

In [ ]:
# Plot categorical feature distributions

plt.figure(figsize=(8, 5))
sns.countplot(y=train_scaled["Loan_Status"], order=train["Loan_Status"].value_counts().index)
plt.title(f"Distribution of Loan_Status")
plt.xlabel("Count")
plt.ylabel("Loan_Status")
plt.show()

In my experience:
When I upsample then split into train test, there is a chance of getting overfit because some of the test values are already seen.

But in this we already have the seperate test data we can upsample here.

In [ ]:
# Separate majority and minority classes
df_majority = train_scaled[train_scaled["Loan_Status"] == 1]
df_minority = train_scaled[train_scaled["Loan_Status"] == 0]

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,  # Sample with replacement
                                 n_samples=len(df_majority),  # Match majority class count
                                 random_state=42)  # Ensure reproducibility

# Combine majority class with upsampled minority class
train_upsample = pd.concat([df_majority, df_minority_upsampled])

# Shuffle the dataset
train_upsample = train_upsample.sample(frac=1, random_state=42).reset_index(drop=True)

# Check class distribution after upsampling
print(train_upsample["Loan_Status"].value_counts())

# Downsample majority class
df_majority_downsampled = resample(df_majority,
                                   replace=False,  # No replacement (randomly remove samples)
                                   n_samples=len(df_minority),  # Match minority class count
                                   random_state=42)  # Ensure reproducibility

# Combine minority class with downsampled majority class
train_downsample = pd.concat([df_majority_downsampled, df_minority])

# Shuffle the dataset
train_downsample = train_downsample.sample(frac=1, random_state=42).reset_index(drop=True)

# Check class distribution after downsampling
print(train_downsample["Loan_Status"].value_counts())


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(y=train_downsample["Loan_Status"], order=train_downsample["Loan_Status"].value_counts().index)
plt.title(f"Distribution of Loan_Status in downsampled")
plt.xlabel("Count")
plt.ylabel("Loan_Status")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(y=train_upsample["Loan_Status"], order=train_upsample["Loan_Status"].value_counts().index)
plt.title(f"Distribution of Loan_Status in upsampled")
plt.xlabel("Count")
plt.ylabel("Loan_Status")
plt.show()

In [ ]:
train_downsample.head()

In [ ]:
train_upsample.head()